# Plotting diurnal cycles

Quick example of how to compute and plot the average diurnal cycle for the average surface temperature of each model. First, we start up a Kubernetes cluster and client:

In [ ]:
from dask_kubernetes import KubeCluster

cluster = KubeCluster(n_workers=10)
cluster

In [ ]:
from dask.distributed import Client

client = Client(cluster)
client

Then we open up the preexisting catalog of TRACMIP data (we will search for the relevant data):

In [ ]:
from intake import open_catalog

cat = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/climate.yaml")
col = cat.tracmip()
col

In [ ]:
col.df

In [ ]:
ds_dict = col.search(frequency="A3hr", experiment="landControl",
                     variable="ts").to_dataset_dict()

Finally, we can use this data to generate a rough plot of the average diurnal cycle over different regions of each model:

In [ ]:
ds_dict.keys()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(18, 20), facecolor='w', edgecolor='k')

for i, source in enumerate(ds_dict.keys()):
    ax = plt.subplot(5, 3, i + 1)
    ds_dict[source]["ts"].sel(lon=slice(0,45), lat=slice(-30,30)) \
                         .groupby("time.hour").mean(dim=("time", "lat", "lon")) \
                         .plot(color="red", label="land")
    ds_dict[source]["ts"].sel(lon=slice(45,360), lat=slice(-30,30)) \
                         .groupby("time.hour").mean(dim=("time", "lat", "lon")) \
                         .plot(color="blue", label="ocean")
    ax.set(title=ds_dict[source].attrs["model_id"], ylabel=ds_dict[source]["ts"].attrs["units"])
    ax.legend()
    
plt.tight_layout()